In [2]:
import h5py
import os
from extratorFeatures import *
import json

In [7]:
PROGRESS_FILE = "Datasets/progressDataset.txt"

def salvar_progresso(caminho_imagem, indice_contorno):
    """Salva o caminho da imagem e o índice do último contorno processado."""
    progresso = {'ultimo_caminho_imagem': caminho_imagem, 'ultimo_indice_contorno': indice_contorno}
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progresso, f)

def carregar_progresso():
    """Carrega o progresso. Se não houver arquivo, começa do início."""
    if not os.path.exists(PROGRESS_FILE):
        return None, -1
    with open(PROGRESS_FILE, 'r') as f:
        try:
            progresso = json.load(f)
            return progresso.get('ultimo_caminho_imagem'), progresso.get('ultimo_indice_contorno', -1)
        except json.JSONDecodeError:
            # Se o arquivo estiver corrompido, começa do zero
            return None, -1

def resetar_progresso():
    """Apaga o arquivo de progresso para recomeçar a anotação do zero."""
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
        print("Progresso de anotação resetado. O script irá começar do início na próxima execução.")
    else:
        print("Nenhum progresso salvo para resetar.")

# --- Função Principal de Anotação ---

def anotar_dataset_interativo(dataset_name='Datasets/dataset_anotado.h5', reset=False):
    """
    Cria ou atualiza um dataset de forma interativa, com capacidade de salvar
    e continuar o progresso.
    """
    if reset:
        resetar_progresso()
        return

    caminho_classe_1 = 'Frames2Treinamento/Fire/'
    caminho_classe_0 = 'Frames2Treinamento/Normal/'
    NUM_FEATURES = 15

    lista_classe_1 = [(os.path.join(caminho_classe_1, f), 'Fire') for f in sorted(os.listdir(caminho_classe_1))]
    lista_classe_0 = [(os.path.join(caminho_classe_0, f), 'Normal') for f in sorted(os.listdir(caminho_classe_0))]
    lista_completa = lista_classe_1 + lista_classe_0

    features_list = []
    labels_list = []

    # Carrega o progresso anterior
    ultimo_caminho, ultimo_contorno_idx = carregar_progresso()
    pular_ate_encontrar = ultimo_caminho is not None

    print("--- INICIANDO ANOTAÇÃO INTERATIVA ---")
    print("  - 's': FUMAÇA (classe 1) | 'n': NÃO FUMAÇA (classe 0)")
    print("  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno")
    print(f"Dataset de saída: {dataset_name}")
    if pular_ate_encontrar:
        print(f"Continuando a partir da imagem: {os.path.basename(ultimo_caminho)}")
    print("-" * 45)

    # Loop principal por todas as imagens
    for i, (caminho_img, tipo_pasta) in enumerate(lista_completa):
        # Lógica para pular imagens já anotadas em sessões anteriores
        if pular_ate_encontrar and caminho_img != ultimo_caminho:
            continue
        elif pular_ate_encontrar and caminho_img == ultimo_caminho:
            pular_ate_encontrar = False # Encontrou, agora começa a processar
            print(f"\nRetomando anotação na imagem: {caminho_img}")
        else:
            print(f"\nProcessando nova imagem {i+1}/{len(lista_completa)}: {caminho_img}")

        img = cv2.imread(caminho_img)
        if img is None: continue

        contornos, img_hsv = extrair_contornos(img)
        if not contornos: continue

        # Loop para anotar cada contorno na imagem
        for j, contorno in enumerate(contornos):
            # Se estamos na imagem onde paramos, pulamos os contornos já anotados
            if not pular_ate_encontrar and ultimo_caminho == caminho_img and j <= ultimo_contorno_idx:
                continue

            img_display = img.copy()
            cv2.drawContours(img_display, contornos, -1, (0, 255, 0), 2)
            cv2.drawContours(img_display, [contorno], -1, (0, 255, 255), 3)

            cv2.namedWindow('Anotador - (s/n/i/q)', cv2.WINDOW_NORMAL)
            cv2.imshow('Anotador - (s/n/i/q)', img_display)
            key = cv2.waitKey(0) & 0xFF

            if key == ord('q'):
                print("\n'q' pressionado. Saindo e salvando...")
                cv2.destroyAllWindows()
                break

            label_to_save = -1
            if key == ord('s'):
                label_to_save = 1
                status = "FUMAÇA (1)"
            elif key == ord('n'):
                label_to_save = 0
                status = "NÃO FUMAÇA (0)"
            elif key == ord('i'):
                print(f"  Contorno {j+1}/{len(contornos)}: Ignorado.")
                salvar_progresso(caminho_img, j) # Salva progresso mesmo ao ignorar
                continue
            else: # Qualquer outra tecla também ignora
                print(f"  Contorno {j+1}/{len(contornos)}: Tecla inválida, ignorado.")
                salvar_progresso(caminho_img, j)
                continue

            features = extrair_features(img_hsv, contorno)
            if features is not None and features.shape[0] == NUM_FEATURES:
                features_list.append(features)
                labels_list.append(label_to_save)
                print(f"  Contorno {j+1}/{len(contornos)}: Salvo como {status}")
            else:
                print(f"  -> Aviso: Falha ao extrair features. Pulando.")

            # Salva o progresso após cada anotação
            salvar_progresso(caminho_img, j)

        cv2.destroyAllWindows()
        if 'key' in locals() and key == ord('q'):
            break

    if not features_list:
        print("\nNenhuma nova amostra foi anotada nesta sessão.")
        return

    X_data = np.array(features_list, dtype=np.float32)
    y_data = np.array(labels_list, dtype=np.int8)

    with h5py.File(dataset_name, 'a') as hf: # 'a' para append (adicionar)
        if 'features' in hf and 'labels' in hf:
            print(f"\nAdicionando {len(X_data)} novas amostras ao dataset existente...")
            # Redimensiona os datasets existentes
            hf['features'].resize(hf['features'].shape[0] + len(X_data), axis=0)
            hf['labels'].resize(hf['labels'].shape[0] + len(y_data), axis=0)
            # Adiciona os novos dados no final
            hf['features'][-len(X_data):] = X_data
            hf['labels'][-len(y_data):] = y_data
        else:
            print(f"\nCriando novo dataset com {len(X_data)} amostras...")
            # Cria os datasets com tamanho máximo expansível
            hf.create_dataset('features', data=X_data, compression="gzip", chunks=True, maxshape=(None, NUM_FEATURES))
            hf.create_dataset('labels', data=y_data, compression="gzip", chunks=True, maxshape=(None,))

    print(f"Operação concluída com sucesso!")
    with h5py.File(dataset_name, 'r') as hf:
        print(f"Dimensões totais do dataset agora: {hf['features'].shape}")

In [8]:
anotar_dataset_interativo("Datasets/dataset_features.h5")

--- INICIANDO ANOTAÇÃO INTERATIVA ---
  - 's': FUMAÇA (classe 1) | 'n': NÃO FUMAÇA (classe 0)
  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno
Dataset de saída: Datasets/dataset_features.h5
Continuando a partir da imagem: frame_00000.jpg
---------------------------------------------

Retomando anotação na imagem: Frames2Treinamento/Fire/frame_00000.jpg
  Contorno 2/26: Salvo como NÃO FUMAÇA (0)
  Contorno 3/26: Salvo como NÃO FUMAÇA (0)
  Contorno 4/26: Salvo como FUMAÇA (1)
  Contorno 5/26: Salvo como FUMAÇA (1)
  Contorno 6/26: Salvo como FUMAÇA (1)
  Contorno 7/26: Salvo como FUMAÇA (1)
  Contorno 8/26: Salvo como FUMAÇA (1)
  Contorno 9/26: Salvo como NÃO FUMAÇA (0)
  Contorno 10/26: Salvo como FUMAÇA (1)
  Contorno 11/26: Salvo como NÃO FUMAÇA (0)
  Contorno 12/26: Salvo como NÃO FUMAÇA (0)
  Contorno 13/26: Salvo como NÃO FUMAÇA (0)
  Contorno 14/26: Salvo como NÃO FUMAÇA (0)
  Contorno 15/26: Salvo como NÃO FUMAÇA (0)
  Contorno 16/26: Salvo como NÃO FUMAÇA (0)
  Contorno 17/26: S